# Nearest Restaurants 

#### Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

#### Read csv file into pandas dataframe that contains the types of restaurants that we'll search for

In [ ]:
types_df = pd.read_csv("../Resources/ethnic_restr.csv")
types_df.head()

#### Add columns to the newly created dataframe that we will use to store additional restaurant information

In [ ]:
types_df['name'] = ""
types_df['address'] = ""
types_df['price_level'] = ""
types_df['rating'] = ""

types_df.head()

#### Setup the framework for the API call

In [ ]:
# Create base API url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Dictionary to store query parameters
params = {
    "location": "29.714033,-95.406480", # coordinates for Anderson-Clarke Center (exactly where we are sitting)
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
}

#### Loop over DataFrame to execute an API call for each restaurant type, then store the appropriate response data in its corresponding column

Upon each iteration of the loop, we'll overwrite `restr_type` and execute a new API request using a new restaurant type.

In [ ]:
# use iterrows to iterate through pandas DataFrame
for index, row in types_df.iterrows():

    # get restaurant type from DataFrame
    restr_type = row['ethnicity']

    # add a new key to our params dictionary that will store our keyword to search for
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # Use exception handling in case any of our restaurants' attributes are missing
    # NOTE: if our code hits a KeyError or IndexError on ANY of the lines of code below, it will skip any unexecuted code and move to the next iteration
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
types_df